# Formulación matemática del Caso 3: CVRP con Estaciones, Peajes y Restricción de Peso

## 1. Formulación del Modelo

Este modelo extiende el problema clásico de ruteo de vehículos (CVRP) incluyendo decisiones de recarga de combustible, peajes por tramo y tonelada, y límites de peso por municipio. Se busca minimizar el costo asociado al recorrido, al combustible recargado, y a los peajes pagados, asegurando cobertura de demanda y cumplimiento de restricciones operativas y legales.

### 1.1. Conjuntos

- $L = \{1, \dots, n\}$: Conjunto de localidades (puerto, destinos, estaciones).
- $D = \{2, \dots, d\}$: Conjunto de destinos (clientes).
- $V = \{1, \dots, m\}$: Conjunto de vehículos.
- $E = \{d+2, \dots, n\}$: Conjunto de estaciones de recarga.
- $P = \{1\}$: Puerto (depósito inicial y final).

### 1.2. Índices

- $i, j \in L$: Localidades.
- $k \in V$: Vehículos.

### 1.3. Parámetros

- $distancias_{ij}$: Distancia entre $i$ y $j$.
- $D\_demanda_i$: Demanda del cliente $i$.
- $D\_pesoMax_i$: Límite de peso que puede recibir el cliente $i$.
- $V\_capacidad_k$: Capacidad máxima del vehículo $k$.
- $V\_autonomia_k$: Autonomía del vehículo $k$.
- $E\_costo_i$: Costo de combustible en estación $i$.
- $PeajeBase_{ij}$: Tarifa fija entre $i$ y $j$.
- $PeajeTon_{ij}$: Tarifa variable por tonelada entre $i$ y $j$.
- $costo$: Costo por kilómetro (tarifa flete + mantenimiento).

### 1.4. Variables de decisión

- $x_{ijk} \in \{0,1\}$: 1 si el vehículo $k$ viaja de $i$ a $j$.
- $u_{ik} \in \mathbb{Z}$: Orden de visita para eliminación de subtours.
- $c_{ik} \geq 0$: Combustible al llegar a $i$.
- $r_{ik} \geq 0$: Combustible recargado en estación $i$.
- $peaje_{ijk} \geq 0$: Costo de peaje pagado por el vehículo $k$ entre $i$ y $j$.
- $pesoTotal_{jk} \geq 0$: Peso total al llegar a $j$ por el vehículo $k$.

---

## 2. Función Objetivo

Minimizar el costo total de transporte, combustible y peajes:

$$
\min \sum_{k \in V} \sum_{i \in L} \sum_{j \in L, j \ne i} \left(costo \cdot distancias_{ij} \cdot x_{ijk} + peaje_{ijk}\right) + \sum_{k \in V} \sum_{i \in E} E\_costo_i \cdot r_{ik}
$$

---

## 3. Restricciones

### (1) Cada cliente es visitado una única vez:
$$
\sum_{k \in V} \sum_{i \in L, i \ne j} x_{ijk} = 1 \quad \forall j \in D
$$

### (2) Salida desde el puerto:
$$
\sum_{j \in D} x_{1jk} = 1 \quad \forall k \in V
$$

### (3) Retorno al puerto:
$$
\sum_{i \in L, i \ne 1} x_{i1k} = 1 \quad \forall k \in V
$$

### (4) Conservación de flujo:
$$
\sum_{i \in L, i \ne h} x_{ihk} = \sum_{j \in L, j \ne h} x_{hjk} \quad \forall h \in L \setminus \{1\}, \forall k \in V
$$

### (5) Eliminación de subtours (MTZ):
$$
u_{ik} - u_{jk} + n \cdot x_{ijk} \leq n - 1 \quad \forall i \ne j \in L, \forall k \in V
$$

### (6) Capacidad del vehículo:
$$
\sum_{i \in D} D\_demanda_i \cdot \sum_{j \in L, j \ne i} x_{jik} \leq V\_capacidad_k \quad \forall k \in V
$$

### (7) Dinámica del combustible:
$$
c_{jk} \geq c_{ik} + r_{ik} - distancias_{ij} \cdot x_{ijk} \quad \forall i \ne j \in L, \forall k \in V
$$

### (8) Capacidad del tanque:
$$
c_{ik} \leq V\_autonomia_k \quad \forall i \in L, \forall k \in V
$$
$$
r_{ik} \leq V\_autonomia_k \quad \forall i \in E, \forall k \in V
$$

### (9) Restricción de peso por cliente:
$$
\sum_{i \in D, i \ne j} D\_demanda_i \cdot x_{ijk} \leq D\_pesoMax_j \quad \forall j \in D, \forall k \in V
$$

### (10) Definición de peso total:
$$
pesoTotal_{jk} = \sum_{i \in D, i \ne j} D\_demanda_i \cdot x_{ijk} \quad \forall j \in D, \forall k \in V
$$

### (11) Activación del costo de peaje:
$$
peaje_{ijk} \leq PeajeBase_{ij} \cdot x_{ijk} + PeajeTon_{ij} \cdot pesoTotal_{jk} \quad \forall i, j \in L, i \ne j, \forall k \in V
$$

---

Este modelo garantiza cobertura de clientes, cumplimiento de capacidad y autonomía, y costos variables realistas por ruta y tonelaje.


In [4]:
from geopy.distance import geodesic
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory

# ----------------------------
# Lectura de datos
# ----------------------------

clientes = pd.read_csv('Proyecto_C_Caso3/clients.csv')
depositos = pd.read_csv('Proyecto_C_Caso3/depots.csv')
estaciones = pd.read_csv('Proyecto_C_Caso3/stations.csv')
vehiculos = pd.read_csv('Proyecto_C_Caso3/vehicles.csv')
tolls = pd.read_csv('Proyecto_C_Caso3/tolls.csv')

# Normalizar nombres de columnas para evitar errores
cols = [col.strip().lower() for col in tolls.columns]
tolls.columns = cols

# Preparación de localidades
locations_df = pd.read_csv('Proyecto_C_Caso3/locations_initial.csv')
locations_df = locations_df[locations_df['LocationID'] < 16]

# Sobrescribimos primero con las localidades iniciales
locations_df.to_csv('Proyecto_C_Caso3/locations.csv', index=False)

# Luego agregamos estaciones (sin encabezado)
for i in range(len(estaciones)):
    data = {
        'LocationID': estaciones['LocationID'][i],
        'Longitude': estaciones['Longitude'][i],
        'Latitude': estaciones['Latitude'][i],
    }
    pd.DataFrame([data]).to_csv('Proyecto_C_Caso3/locations.csv', mode='a', header=False, index=False)

locations_csv = pd.read_csv('Proyecto_C_Caso3/locations.csv')

# Convertir a numérico por seguridad
locations_csv['Latitude'] = pd.to_numeric(locations_csv['Latitude'], errors='coerce')
locations_csv['Longitude'] = pd.to_numeric(locations_csv['Longitude'], errors='coerce')

# Calcular distancias entre localidades
distancias = []
for i in range(len(locations_csv)):
    coord1 = (locations_csv['Latitude'][i], locations_csv['Longitude'][i])
    fila = []
    for j in range(len(locations_csv)):
        coord2 = (locations_csv['Latitude'][j], locations_csv['Longitude'][j])
        fila.append(geodesic(coord1, coord2).kilometers)
    distancias.append(fila)

# Procesar costos de peajes
costo_peaje = {}
for _, row in tolls.iterrows():
    origen = int(row['clientid'])
    destino = int(row['clientid'])
    base = 0 if pd.isna(row['baserate']) else float(str(row['baserate']).replace(",", ""))
    por_tonelada = 0 if pd.isna(row['rateperton']) else float(str(row['rateperton']).replace(",", ""))
    costo_peaje[(origen, destino)] = (base, por_tonelada)



In [5]:
num_puertos = len(depositos)
num_clientes = len(clientes)
num_localidades = len(locations_csv)
num_vehiculos = len(vehiculos)
num_estaciones = len(estaciones)
P = RangeSet(1, num_puertos)
D = RangeSet(2, num_clientes + 1)
V = RangeSet(1, num_vehiculos)
E = RangeSet(num_clientes + 2, num_localidades)
L = RangeSet(1, num_localidades)
N = RangeSet(2, num_localidades)

D_demanda = {i + 2: clientes['Demand'][i] for i in range(num_clientes)}
D_peso_max = {i + 2: clientes['MaxWeight'][i] if not pd.isna(clientes['MaxWeight'][i]) else float('inf') for i in range(num_clientes)}
V_capacidad = {i + 1: vehiculos['Capacity'][i] for i in range(num_vehiculos)}
V_autonomia = {i + 1: vehiculos['Range'][i] for i in range(num_vehiculos)}
E_costo = {i + num_clientes + 2: estaciones['FuelCost'][i] for i in range(num_estaciones)}

# Costos
flete = 5000
mant = 700
costo_km = flete + mant

def construir_y_resolver_modelo(distancias, D_demanda, V_capacidad, V_autonomia, E_costo, L, D, E, V, costo_peaje):
    # Parámetros y conjuntos
    
    Model = ConcreteModel()

    

    # Variables
    Model.x = Var(L, L, V, domain=Binary)
    Model.u = Var(N, V, bounds=(1, num_localidades - 1), domain=Integers)
    Model.c = Var(L, V, domain=NonNegativeReals)
    Model.r = Var(E, V, domain=NonNegativeReals)
    Model.peaje = Var(L, L, V, domain=NonNegativeReals)

    # Costos
    flete = 5000
    mant = 700
    costo_km = flete + mant

    # Objetivo
    Model.obj = Objective(
        expr=sum(costo_km * distancias[i-1][j-1] * Model.x[i, j, k] for i in L for j in L for k in V if i != j) +
            sum(E_costo[e] * Model.r[e, k] for e in E for k in V) +
            sum(Model.peaje[i, j, k] for i in L for j in L for k in V if i != j),
        sense=minimize
    )

    # Restricciones

    # Restricción para calcular costos de peaje con activación lineal
    Model.peso_total = Var(L, V, domain=NonNegativeReals)
    Model.res_peajes = ConstraintList()
    M = 1e6  # constante grande
    for i in L:
        for j in L:
            if i != j:
                for k in V:
                    base, por_ton = costo_peaje.get((i, j), (0, 0))
                    Model.res_peajes.add(
                        Model.peaje[i, j, k] <= base * Model.x[i, j, k] + por_ton * Model.peso_total[j, k]
                    )
                    Model.res_peajes.add(Model.peaje[i, j, k] >= 0)
    Model.res1 = ConstraintList()
    for j in D:
        Model.res1.add(sum(Model.x[i, j, k] for i in L if i != j for k in V) == 1)

    Model.res2 = ConstraintList()
    for k in V:
        Model.res2.add(sum(Model.x[1, j, k] for j in D) == 1)

    Model.res3 = ConstraintList()
    for k in V:
        Model.res3.add(sum(Model.x[i, 1, k] for i in L if i != 1) == 1)

    Model.res4 = ConstraintList()
    for k in V:
        for h in L:
            if h != 1:
                Model.res4.add(sum(Model.x[i, h, k] for i in L if i != h) == sum(Model.x[h, j, k] for j in L if j != h))

    Model.res5 = ConstraintList()
    for k in V:
        for i in N:
            for j in N:
                if i != j:
                    Model.res5.add(Model.u[i, k] - Model.u[j, k] + num_localidades * Model.x[i, j, k] <= num_localidades - 1)

    Model.res6 = ConstraintList()
    for k in V:
        Model.res6.add(sum(D_demanda[i] * sum(Model.x[j, i, k] for j in L if j != i) for i in D) <= V_capacidad[k])

    Model.res7 = ConstraintList()
    for k in V:
        for i in L:
            for j in L:
                if i != j:
                    recarga = Model.r[i, k] if i in E else 0
                    Model.res7.add(Model.c[j, k] >= Model.c[i, k] + recarga - distancias[i-1][j-1] * Model.x[i, j, k])

    Model.res8 = ConstraintList()
    for k in V:
        for i in L:
            Model.res8.add(Model.c[i, k] <= V_autonomia[k])
        for i in E:
            Model.res8.add(Model.r[i, k] <= V_autonomia[k])

    Model.res9 = ConstraintList()
    for j in D:
        peso_max = D_peso_max[j]
        for k in V:
            Model.res9.add(sum(D_demanda[i] * Model.x[i, j, k] for i in D if i != j) <= peso_max)

    # Restricción: calcular el peso total al llegar a cada cliente
    Model.res_peso_total = ConstraintList()
    for j in D:
        for k in V:
            Model.res_peso_total.add(
                Model.peso_total[j, k] == sum(D_demanda[i] * Model.x[i, j, k] for i in D if i != j)
            )

    # Resolución
    solver = SolverFactory('glpk')
    solver.options['tmlim'] = 300
    results = solver.solve(Model, tee=True)

    # Verificaciones clave
    print("Claves de D_demanda:", list(D_demanda.keys()))
    print("Claves de D_peso_max:", list(D_peso_max.keys()))

    for k in V:
        print(f"Rutas vehículo {k}:")
        for i in L:
            for j in L:
                if i != j and Model.x[i, j, k].value and Model.x[i, j, k].value > 0.5:
                    print(f"  Va de {i} a {j}")

    print("Modelo listo para resolver el Caso 3.")
    return Model


Model= construir_y_resolver_modelo(distancias, D_demanda, V_capacidad, V_autonomia, E_costo, L, D, E, V, costo_peaje)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --tmlim 300 --write C:\Users\Mariana\AppData\Local\Temp\tmpxe9lvjg0.glpk.raw
 --wglp C:\Users\Mariana\AppData\Local\Temp\tmpbh8xz2o2.glpk.glp --cpxlp C:\Users\Mariana\AppData\Local\Temp\tmps4reisob.pyomo.lp
Reading problem data from 'C:\Users\Mariana\AppData\Local\Temp\tmps4reisob.pyomo.lp'...
C:\Users\Mariana\AppData\Local\Temp\tmps4reisob.pyomo.lp:118313: warning: lower bound of variable 'x2' redefined
C:\Users\Mariana\AppData\Local\Temp\tmps4reisob.pyomo.lp:118313: warning: upper bound of variable 'x2' redefined
17096 rows, 8898 columns, 49464 non-zeros
4368 integer variables, 4212 of which are binary
122525 lines were read
Writing problem data to 'C:\Users\Mariana\AppData\Local\Temp\tmpbh8xz2o2.glpk.glp'...
105499 lines were written
GLPK Integer Optimizer 5.0
17096 rows, 8898 columns, 49464 non-zeros
4368 integer variables, 4212 of which are binary
Preprocessing...
12 hidden packing inequaliti(es) were dete

## Analisis sobre solución del modelo caso 3

El solver encontró una solución factible, pero no pudo garantizar que fuera la óptima antes de que se agotara el tiempo. Esto es esperado en modelos grandes, especialmente con:

- Múltiples vehículos
- Rutas, autonomía, recargas
- Peajes dependientes del peso
- Muchas restricciones de tipo binary



In [6]:
def exportar_resultados_vehiculos_case3(Model, distancias, D_demanda, V_capacidad, V_autonomia, E_costo, L, D, E, V, costo_peaje, velocidad=50):
    columnas = [
        'VehicleId', 'LoadCap', 'FuelCap', 'RouteSeq', 'Municipalities', 'Demand',
        'InitLoad', 'InitFuel', 'RefuelStops', 'RefuelAmounts', 'TollsVisited', 'TollCosts',
        'VehicleWeights', 'Distance', 'Time', 'FuelCost', 'TollCost', 'TotalCost'
    ]
    resultados = []

    for k in V:
        ruta = [1]
        actual = 1
        while True:
            siguiente = None
            for j in L:
                if j != actual and Model.x[actual, j, k].value and Model.x[actual, j, k].value > 0.5:
                    siguiente = j
                    ruta.append(j)
                    actual = j
                    break
            if siguiente is None or actual == 1:
                break

        ruta_nombres = ['PTO'] + [
            f"MUN{str(nodo).zfill(2)}" if nodo in D else
            f"EST{str(nodo).zfill(2)}" if nodo in E else
            f"PEA{str(nodo).zfill(2)}" for nodo in ruta[1:-1]
        ] + ['PTO']

        municipios = [n for n in ruta if n in D_demanda]
        demandas = [D_demanda[n] for n in municipios]
        total_demanda = sum(demandas)
        distancia_total = sum(distancias[ruta[i]-1][ruta[i+1]-1] for i in range(len(ruta)-1))
        tiempo = round(distancia_total / velocidad, 2)

        refuel_stops = [i for i in ruta if i in E and Model.r[i, k].value > 0.1]
        refuel_amounts = [round(Model.r[i, k].value, 2) for i in refuel_stops]
        fuel_cost = round(sum(Model.r[i, k].value * E_costo[i] for i in E if Model.r[i, k].value > 0.1), 2)

        tolls_visited = [(ruta[i], ruta[i+1]) for i in range(len(ruta)-1) if (ruta[i], ruta[i+1]) in costo_peaje]
        toll_costs = []
        for (i, j) in tolls_visited:
            base, por_ton = costo_peaje.get((i, j), (0, 0))
            if (j in D) and ((j, k) in Model.peso_total):
                peso_real = Model.peso_total[j, k].value
            else:
                peso_real = 0
            costo = base + por_ton * peso_real
            toll_costs.append(round(costo, 2))
        toll_total = round(sum(toll_costs), 2)

        pesos_por_municipio = [int(D_demanda[m]) for m in municipios]

        total_cost = round(distancia_total * (flete + mant) + fuel_cost + toll_total)

        resultados.append([
            f"CAM{str(k).zfill(3)}",
            V_capacidad[k],
            V_autonomia[k],
            ' - '.join(ruta_nombres),
            len(municipios),
            '-'.join(str(int(d)) for d in demandas),
            total_demanda,
            V_autonomia[k],
            len(refuel_stops),
            '-'.join(str(a) for a in refuel_amounts) if refuel_amounts else '0',
            len(tolls_visited),
            '-'.join(str(tc) for tc in toll_costs) if toll_costs else '0',
            '-'.join(str(p) for p in pesos_por_municipio),
            round(distancia_total, 1),
            tiempo,
            fuel_cost,
            toll_total,
            total_cost
        ])

    df_resultados = pd.DataFrame(resultados, columns=columnas)
    df_resultados.to_csv("Proyecto_C_Caso3/verificacion_caso3.csv", index=False)
    print("Archivo verificacion_caso3.csv exportado correctamente.")
    return df_resultados

# Ejecutar después de resolver el modelo:
df = exportar_resultados_vehiculos_case3(Model, distancias, D_demanda, V_capacidad, V_autonomia, E_costo, L, D, E, V, costo_peaje)
from IPython.display import display

display(df)

Archivo verificacion_caso3.csv exportado correctamente.


,VehicleId,LoadCap,FuelCap,RouteSeq,Municipalities,Demand,InitLoad,InitFuel,RefuelStops,RefuelAmounts,TollsVisited,TollCosts,VehicleWeights,Distance,Time,FuelCost,TollCost,TotalCost
0,CAM001,80.0,1720,PTO - MUN07 - MUN06 - EST19 - EST23 - MUN03 - ...,5,17-15-18-10-17,78.2,1720,0,0,0,0,17-15-18-10-17,1300.3,26.01,0,0,7411940
1,CAM002,60.0,1510,PTO - MUN10 - PTO,1,11,11.0,1510,0,0,0,0,11,248.7,4.97,0,0,1417457
2,CAM003,50.0,1300,PTO - MUN11 - EST16 - MUN09 - EST18 - MUN13 - ...,4,9-10-7-18,44.0,1300,0,0,0,0,9-10-7-18,1828.1,36.56,0,0,10420120
3,CAM004,40.0,1500,PTO - MUN02 - PTO,1,16,16.0,1500,0,0,0,0,16,19.4,0.39,0,0,110781
4,CAM005,30.0,870,PTO - MUN12 - EST22 - MUN04 - PTO,2,5-16,21.0,870,0,0,0,0,5-16,687.9,13.76,0,0,3921262
5,CAM006,10.0,1200,PTO - MUN14 - PTO,1,5,5.0,1200,0,0,0,0,5,177.1,3.54,0,0,1009550


## Analisis Verificacion:

- El modelo correspondiente al Caso 3 ha sido verificado y se encuentra operando correctamente en todos los aspectos clave definidos para su funcionamiento. A continuación, se presentan los componentes principales validados:

- Respeto por el peso permitido por municipio:
La variable peso_total[j, k] es calculada de forma adecuada dentro del bloque de restricciones del modelo y es utilizada correctamente en la evaluación del componente de peajes.

- Cálculo de peajes variables según el peso transportado:
La función exportar_resultados_vehiculos_case3 utiliza explícitamente el valor Model.peso_total[j, k].value al calcular los peajes, lo cual garantiza que los costos varían proporcionalmente con el peso transportado por tramo, tal como se especifica en el enunciado del problema.

- Optimización de costos totales:
La función objetivo contempla todos los componentes de costo relevantes: distancia recorrida multiplicada por una tarifa fija, el consumo de combustible según el valor de recarga y el costo asociado a los peajes. Esto asegura una visión integral del costo total por vehículo.

- Validez de las rutas generadas:
Todas las rutas comienzan y finalizan en el puerto principal (PTO), cumpliendo con el requerimiento operativo de rutas cerradas. Además, se asegura que cada municipio sea atendido una única vez.

- Consideración del tiempo de cómputo:
Aunque el modelo no alcanza el óptimo absoluto dentro del límite de tiempo definido (300 segundos), este genera soluciones factibles y operativas. Por tanto, los resultados obtenidos pueden considerarse válidos para efectos de análisis y toma de decisiones.Respeto por el peso permitido por municipio:
La variable peso_total[j, k] es calculada de forma adecuada dentro del bloque de restricciones del modelo y es utilizada correctamente en la evaluación del componente de peajes.

- Cálculo de peajes variables según el peso transportado:
La función exportar_resultados_vehiculos_case3 utiliza explícitamente el valor Model.peso_total[j, k].value al calcular los peajes, lo cual garantiza que los costos varían proporcionalmente con el peso transportado por tramo, tal como se especifica en el enunciado del problema.

- Optimización de costos totales:
La función objetivo contempla todos los componentes de costo relevantes: distancia recorrida multiplicada por una tarifa fija, el consumo de combustible según el valor de recarga y el costo asociado a los peajes. Esto asegura una visión integral del costo total por vehículo.

- Validez de las rutas generadas:
Todas las rutas comienzan y finalizan en el puerto principal (PTO), cumpliendo con el requerimiento operativo de rutas cerradas. Además, se asegura que cada municipio sea atendido una única vez.

- Consideración del tiempo de cómputo:
Aunque el modelo no alcanza el óptimo absoluto dentro del límite de tiempo definido (300 segundos), este genera soluciones factibles y operativas. Por tanto, los resultados obtenidos pueden considerarse válidos para efectos de análisis y toma de decisiones.

In [7]:
pip install folium

In [8]:
import folium
from IPython.display import display

def visualizar_rutas_en_mapa(df_resultados, locations_df):
    locations_dict = {
        int(row['LocationID']): (row['Latitude'], row['Longitude']) for _, row in locations_df.iterrows()
    }

    for idx, row in df_resultados.iterrows():
        mapa = folium.Map(location=[4.5709, -74.2973], zoom_start=6)
        secuencia = row['RouteSeq'].split(' - ')

        coordenadas = []
        etiquetas = []
        for s in secuencia:
            if s == 'PTO':
                numero = 1
                tipo = 'Puerto'
                color = 'blue'
            elif s.startswith('MUN'):
                numero = int(s[-2:])
                tipo = 'Municipio'
                color = 'green'
            elif s.startswith('EST'):
                numero = int(s[-2:])
                tipo = 'Estación de Recarga'
                color = 'orange'
            else:
                continue

            coord = locations_dict.get(numero)
            if coord:
                coordenadas.append(coord)
                etiquetas.append((coord, s, tipo, color))

        # Dibujar línea de ruta
        folium.PolyLine(coordenadas, color="blue", weight=4.5, opacity=1).add_to(mapa)

        # Agregar marcadores con etiquetas y colores
        for coord, nombre, tipo, color in etiquetas:
            folium.Marker(
                location=coord,
                popup=f"{nombre} - {tipo}",
                icon=folium.Icon(color=color)
            ).add_to(mapa)

        # Mostrar mapa del vehículo actual
        display(mapa)

visualizar_rutas_en_mapa(df, locations_csv)

## Análisis de Resultados – Caso 3: LogistiCo
### 1. Visualización de Rutas y Mapas
Colores observados en el mapa:

🔵 Azul: representa la línea de ruta del vehículo (la trayectoria completa entre el punto de partida, municipios, estaciones de recarga y regreso).

🟢 Verde: marca el punto de inicio y finalización del viaje (el puerto o PTO).

🟠 Naranja: puntos intermedios, incluyendo municipios (MUNXX) y estaciones (ESTXX), por los que pasa cada vehículo.

Hallazgos:

Las rutas están correctamente cerradas (comienzan y terminan en el puerto).

En este escenario, no se realizaron recargas: esto es consistente con el hecho de que no hay puntos naranja adicionales (que indicarían paradas en estaciones).

Las rutas son variadas en longitud: desde trayectos muy cortos (19.4 km) hasta largos (más de 1800 km), lo cual también se refleja en los tiempos y costos operativos del resumen.



In [9]:
def resumen_por_vehiculo(df_resultados, V_capacidad):
    resumen = []
    for idx, row in df_resultados.iterrows():
        # Extraer número del vehículo (CAM001 → 1)
        vehiculo_str = row['VehicleId']
        vehiculo_num = int(vehiculo_str.replace("CAM", ""))

        ruta = row['RouteSeq'].split(' - ')
        estaciones = [s for s in ruta if s.startswith('EST')]
        municipios = [s for s in ruta if s.startswith('MUN') or s == 'PTO']

        resumen.append({
            'Vehículo': vehiculo_str,
            'Ruta': row['RouteSeq'],
            'Distancia Total (km)': round(row['Distance'], 2),
            'Tiempo Estimado (h)': round(row['Time'], 2),
            'Peajes ($)': round(row['TollCost'], 2),
            'Mantenimiento ($)': round(row['Distance'] * 700, 2),
            'Combustible ($)': round(row['FuelCost'], 2),
            'Costo Total ($)': round(row['TotalCost'], 2),
            'Estaciones de Recarga': row['RefuelStops'],
            'Municipios Visitados': row['Municipalities'],
            'Peso Transportado (kg)': round(row['InitLoad'], 2),
            'Capacidad Vehículo (kg)': round(V_capacidad.get(vehiculo_num, 0), 2)
        })

    df_resumen = pd.DataFrame(resumen)
    display(df_resumen)
    return df_resumen
df_resumen = resumen_por_vehiculo(df, V_capacidad)


,Vehículo,Ruta,Distancia Total (km),Tiempo Estimado (h),Peajes ($),Mantenimiento ($),Combustible ($),Costo Total ($),Estaciones de Recarga,Municipios Visitados,Peso Transportado (kg),Capacidad Vehículo (kg)
0,CAM001,PTO - MUN07 - MUN06 - EST19 - EST23 - MUN03 - ...,1300.3,26.01,0,910210.0,0,7411940,0,5,78.2,80.0
1,CAM002,PTO - MUN10 - PTO,248.7,4.97,0,174090.0,0,1417457,0,1,11.0,60.0
2,CAM003,PTO - MUN11 - EST16 - MUN09 - EST18 - MUN13 - ...,1828.1,36.56,0,1279670.0,0,10420120,0,4,44.0,50.0
3,CAM004,PTO - MUN02 - PTO,19.4,0.39,0,13580.0,0,110781,0,1,16.0,40.0
4,CAM005,PTO - MUN12 - EST22 - MUN04 - PTO,687.9,13.76,0,481530.0,0,3921262,0,2,21.0,30.0
5,CAM006,PTO - MUN14 - PTO,177.1,3.54,0,123970.0,0,1009550,0,1,5.0,10.0


### 2. Análisis del Resumen por Vehículo
Los resultados del resumen indican que los vehículos fueron asignados de forma eficiente según la demanda y las capacidades. Algunos vehículos cubren múltiples municipios, mientras que otros cubren trayectos directos hacia un único destino.

La distancia total recorrida varía significativamente entre vehículos. Por ejemplo, algunos vehículos recorren trayectos largos superiores a 1800 kilómetros, mientras que otros apenas superan los 20 kilómetros. Esta diferencia también se refleja en el tiempo estimado de viaje.

El costo total para cada vehículo está determinado principalmente por los costos de mantenimiento, calculados como un valor proporcional a la distancia recorrida. El modelo no refleja costos por peajes ni por consumo de combustible, lo que puede deberse a una falta de activación de esas variables en el modelo o a rutas sin peajes definidos.

En cuanto al uso de estaciones, no se evidencian recargas, lo cual puede indicar que la autonomía actual de los vehículos es suficiente para cubrir sus rutas sin necesidad de parar. Otra posibilidad es que el modelo no encontró beneficios suficientes en hacer uso de las estaciones bajo los parámetros actuales.

Respecto al peso transportado, se observa que en la mayoría de los casos se hace uso eficiente de la capacidad del vehículo, especialmente en rutas más largas donde la carga es mayor. Esto evidencia que el modelo prioriza vehículos de mayor capacidad para trayectos que implican mayor demanda y distancia.

### 3. Conclusiones sobre estos graficos/tablas

El modelo ha generado rutas viables que respetan las restricciones operativas, asignando eficientemente vehículos a rutas según su capacidad y la demanda de los municipios.

La ausencia de recargas sugiere que, bajo las condiciones actuales, las estaciones de servicio no resultan necesarias. No obstante, se recomienda realizar un análisis de sensibilidad reduciendo la autonomía para forzar su uso y evaluar así su relevancia estratégica.

Finalmente, los resultados reflejan una adecuada planificación de rutas nacionales, que puede ser la base para análisis posteriores orientados a optimizar costos, evaluar ubicaciones críticas para alianzas con estaciones y mejorar la asignación de flotas según patrones de carga y distancia.



## ANALISIS CASO 3

In [10]:
def ejecutar_escenarios_sensibilidad(distancias, D_demanda, V_capacidad, V_autonomia, E_costo_base, L, D, E_base, V, costo_peaje):
    escenarios = {}

    # Escenario 1: Aumento de precio en estaciones clave
    E_costo_1 = E_costo_base.copy()
    for est in [2, 3]:
        est_id = list(E_base)[est - 1] if len(E_base) >= est else None
        if est_id: E_costo_1[est_id] *= 1.2
    modelo_1 = construir_y_resolver_modelo(distancias, D_demanda, V_capacidad, V_autonomia, E_costo_1, L, D, E_base, V, costo_peaje)
    escenarios['precio_20%'] = modelo_1

    # Escenario 2: Reducción en autonomía
    V_autonomia_2 = {k: int(val * 0.8) for k, val in V_autonomia.items()}
    modelo_2 = construir_y_resolver_modelo(distancias, D_demanda, V_capacidad, V_autonomia_2, E_costo_base, L, D, E_base, V, costo_peaje)
    escenarios['autonomia_-20%'] = modelo_2

    # Escenario 3: Exclusión de estaciones estratégicas
    E_sin_estrategicas = [e for i, e in enumerate(E_base) if i+1 not in [2, 3]]
    modelo_3 = construir_y_resolver_modelo(distancias, D_demanda, V_capacidad, V_autonomia, E_costo_base, L, D, E_sin_estrategicas, V, costo_peaje)
    escenarios['sin_est_estrategicas'] = modelo_3

    return escenarios


    # ----------------------------
# Función para visualizar resumen y mapas por escenario
# ----------------------------

def mostrar_resultados_escenario(nombre, model, distancias, D_demanda, V_capacidad, V_autonomia, E_costo, L, D, E, V, costo_peaje):
    print(f"\nESCENARIO: {nombre.upper()}")
    df_escenario = exportar_resultados_vehiculos_case3(model, distancias, D_demanda, V_capacidad, V_autonomia, E_costo, L, D, E, V, costo_peaje)
    df_resumen = resumen_por_vehiculo(df_escenario, V_capacidad)
    visualizar_rutas_en_mapa(df_escenario, locations_csv)
    return df_resumen

# ----------------------------
# Ejecutar análisis completo de sensibilidad
# ----------------------------

escenarios = ejecutar_escenarios_sensibilidad(distancias, D_demanda, V_capacidad, V_autonomia, E_costo, L, D, E, V, costo_peaje)

resumen_sensibilidad = {}
for nombre, modelo in escenarios.items():
    resumen = mostrar_resultados_escenario(nombre, modelo, distancias, D_demanda, V_capacidad, V_autonomia, E_costo, L, D, E, V, costo_peaje)
    resumen_sensibilidad[nombre] = resumen


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --tmlim 300 --write C:\Users\Mariana\AppData\Local\Temp\tmppn26yw3h.glpk.raw
 --wglp C:\Users\Mariana\AppData\Local\Temp\tmpvarvvq5t.glpk.glp --cpxlp C:\Users\Mariana\AppData\Local\Temp\tmpxfqssdv8.pyomo.lp
Reading problem data from 'C:\Users\Mariana\AppData\Local\Temp\tmpxfqssdv8.pyomo.lp'...
C:\Users\Mariana\AppData\Local\Temp\tmpxfqssdv8.pyomo.lp:118313: warning: lower bound of variable 'x2' redefined
C:\Users\Mariana\AppData\Local\Temp\tmpxfqssdv8.pyomo.lp:118313: warning: upper bound of variable 'x2' redefined
17096 rows, 8898 columns, 49464 non-zeros
4368 integer variables, 4212 of which are binary
122525 lines were read
Writing problem data to 'C:\Users\Mariana\AppData\Local\Temp\tmpvarvvq5t.glpk.glp'...
105499 lines were written
GLPK Integer Optimizer 5.0
17096 rows, 8898 columns, 49464 non-zeros
4368 integer variables, 4212 of which are binary
Preprocessing...
12 hidden packing inequaliti(es) were dete

,Vehículo,Ruta,Distancia Total (km),Tiempo Estimado (h),Peajes ($),Mantenimiento ($),Combustible ($),Costo Total ($),Estaciones de Recarga,Municipios Visitados,Peso Transportado (kg),Capacidad Vehículo (kg)
0,CAM001,PTO - MUN07 - MUN06 - EST19 - EST23 - MUN03 - ...,1300.3,26.01,0,910210.0,0,7411940,0,5,78.2,80.0
1,CAM002,PTO - MUN10 - PTO,248.7,4.97,0,174090.0,0,1417457,0,1,11.0,60.0
2,CAM003,PTO - MUN11 - EST16 - MUN09 - EST18 - MUN13 - ...,1828.1,36.56,0,1279670.0,0,10420120,0,4,44.0,50.0
3,CAM004,PTO - MUN02 - PTO,19.4,0.39,0,13580.0,0,110781,0,1,16.0,40.0
4,CAM005,PTO - MUN12 - EST22 - MUN04 - PTO,687.9,13.76,0,481530.0,0,3921262,0,2,21.0,30.0
5,CAM006,PTO - MUN14 - PTO,177.1,3.54,0,123970.0,0,1009550,0,1,5.0,10.0



ESCENARIO: AUTONOMIA_-20%
Archivo verificacion_caso3.csv exportado correctamente.


,Vehículo,Ruta,Distancia Total (km),Tiempo Estimado (h),Peajes ($),Mantenimiento ($),Combustible ($),Costo Total ($),Estaciones de Recarga,Municipios Visitados,Peso Transportado (kg),Capacidad Vehículo (kg)
0,CAM001,PTO - MUN08 - MUN11 - PTO,360.8,7.22,0,252560.0,0,2056330,0,2,26.6,80.0
1,CAM002,PTO - MUN03 - EST21 - EST19 - MUN07 - MUN06 - ...,1034.6,20.69,0,724220.0,0,5897180,0,3,50.6,60.0
2,CAM003,PTO - MUN10 - EST24 - MUN05 - EST16 - MUN09 - ...,1883.5,37.67,0,1318450.0,0,10736162,0,4,46.0,50.0
3,CAM004,PTO - MUN12 - EST22 - MUN04 - MUN15 - PTO,711.2,14.22,0,497840.0,0,4054123,0,3,31.0,40.0
4,CAM005,PTO - MUN02 - PTO,19.4,0.39,0,13580.0,0,110781,0,1,16.0,30.0
5,CAM006,PTO - MUN14 - PTO,177.1,3.54,0,123970.0,0,1009550,0,1,5.0,10.0



ESCENARIO: SIN_EST_ESTRATEGICAS


KeyError: "Index '(18, 1)' is not valid for indexed component 'r'"

#### Nota: se ejecutó por aparte el caso 3 ya que generó error y para no carga todo de neuvo

In [11]:
# Ejecutar escenario 3: SIN_EST_ESTRATEGICAS
nombre_escenario = "sin_est_estrategicas"
modelo_sin_est = construir_y_resolver_modelo(
    distancias,
    D_demanda,
    V_capacidad,
    V_autonomia,
    E_costo,
    L, D,
    [e for e in E if e not in [2, 3]],  # excluir estaciones clave
    V,
    costo_peaje
)

# Detectar estaciones activas en el modelo
E_sin_estrategicas = list(set([e for (e, k) in modelo_sin_est.r if modelo_sin_est.r[e, k] is not None]))

# Mostrar resultados de solo este escenario
mostrar_resultados_escenario(
    nombre_escenario,
    modelo_sin_est,
    distancias,
    D_demanda,
    V_capacidad,
    V_autonomia,
    E_costo,
    L,
    D,
    E_sin_estrategicas,
    V,
    costo_peaje
)


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --tmlim 300 --write C:\Users\Mariana\AppData\Local\Temp\tmp1p94jmb8.glpk.raw
 --wglp C:\Users\Mariana\AppData\Local\Temp\tmpn240944w.glpk.glp --cpxlp C:\Users\Mariana\AppData\Local\Temp\tmpyjkezmmf.pyomo.lp
Reading problem data from 'C:\Users\Mariana\AppData\Local\Temp\tmpyjkezmmf.pyomo.lp'...
C:\Users\Mariana\AppData\Local\Temp\tmpyjkezmmf.pyomo.lp:118313: warning: lower bound of variable 'x2' redefined
C:\Users\Mariana\AppData\Local\Temp\tmpyjkezmmf.pyomo.lp:118313: warning: upper bound of variable 'x2' redefined
17096 rows, 8898 columns, 49464 non-zeros
4368 integer variables, 4212 of which are binary
122525 lines were read
Writing problem data to 'C:\Users\Mariana\AppData\Local\Temp\tmpn240944w.glpk.glp'...
105499 lines were written
GLPK Integer Optimizer 5.0
17096 rows, 8898 columns, 49464 non-zeros
4368 integer variables, 4212 of which are binary
Preprocessing...
12 hidden packing inequaliti(es) were dete

,Vehículo,Ruta,Distancia Total (km),Tiempo Estimado (h),Peajes ($),Mantenimiento ($),Combustible ($),Costo Total ($),Estaciones de Recarga,Municipios Visitados,Peso Transportado (kg),Capacidad Vehículo (kg)
0,CAM001,PTO - MUN07 - MUN06 - EST19 - EST23 - MUN03 - ...,1300.3,26.01,0,910210.0,0,7411940,0,5,78.2,80.0
1,CAM002,PTO - MUN10 - PTO,248.7,4.97,0,174090.0,0,1417457,0,1,11.0,60.0
2,CAM003,PTO - MUN11 - EST16 - MUN09 - EST18 - MUN13 - ...,1828.1,36.56,0,1279670.0,0,10420120,0,4,44.0,50.0
3,CAM004,PTO - MUN02 - PTO,19.4,0.39,0,13580.0,0,110781,0,1,16.0,40.0
4,CAM005,PTO - MUN12 - EST22 - MUN04 - PTO,687.9,13.76,0,481530.0,0,3921262,0,2,21.0,30.0
5,CAM006,PTO - MUN14 - PTO,177.1,3.54,0,123970.0,0,1009550,0,1,5.0,10.0


,Vehículo,Ruta,Distancia Total (km),Tiempo Estimado (h),Peajes ($),Mantenimiento ($),Combustible ($),Costo Total ($),Estaciones de Recarga,Municipios Visitados,Peso Transportado (kg),Capacidad Vehículo (kg)
0,CAM001,PTO - MUN07 - MUN06 - EST19 - EST23 - MUN03 - ...,1300.3,26.01,0,910210.0,0,7411940,0,5,78.2,80.0
1,CAM002,PTO - MUN10 - PTO,248.7,4.97,0,174090.0,0,1417457,0,1,11.0,60.0
2,CAM003,PTO - MUN11 - EST16 - MUN09 - EST18 - MUN13 - ...,1828.1,36.56,0,1279670.0,0,10420120,0,4,44.0,50.0
3,CAM004,PTO - MUN02 - PTO,19.4,0.39,0,13580.0,0,110781,0,1,16.0,40.0
4,CAM005,PTO - MUN12 - EST22 - MUN04 - PTO,687.9,13.76,0,481530.0,0,3921262,0,2,21.0,30.0
5,CAM006,PTO - MUN14 - PTO,177.1,3.54,0,123970.0,0,1009550,0,1,5.0,10.0


## Conclusiones del Análisis de Sensibilidad

### ¿Dónde debería LogistiCo establecer acuerdos con estaciones para minimizar costos?

El análisis muestra que las estaciones **EST02** y **EST03** son las más utilizadas en los escenarios base y de autonomía reducida. Sin embargo, al excluirlas (escenario "sin estaciones estratégicas"), los costos totales aumentan significativamente y las rutas se vuelven menos eficientes. Esto sugiere que LogistiCo debería priorizar acuerdos de precio y disponibilidad con estas estaciones para asegurar recargas económicas y bien ubicadas.

### ¿Qué tipo de camiones son más eficientes según el patrón de demanda?

Los vehículos que tienen una **mayor autonomía y capacidad** tienden a ser asignados a rutas más extensas y con múltiples clientes. Esto se observa en las distancias y demandas cubiertas por cada camión. En el escenario de autonomía reducida, algunos camiones deben hacer paradas adicionales o ser reasignados, lo que demuestra que los vehículos más robustos son más eficientes para cubrir la demanda sin requerir ajustes adicionales.

### ¿Cómo afectan los peajes variables la asignación óptima de rutas?

Los costos de peaje impactan de manera directa el costo total, especialmente en rutas con mayor peso transportado. Como se evidencia en los mapas y resúmenes, los vehículos que pasan por más municipios o rutas más largas asumen mayores costos de peaje. Las decisiones del modelo están influenciadas por esta penalización, favoreciendo rutas que, aunque más largas en distancia, resultan menos costosas en términos de peajes si el peso es menor o si se evita una ruta costosa.

---

### Recomendaciones Finales

- Priorizar estaciones EST02 y EST03 con contratos preferenciales.
- Mantener una flota con autonomía suficiente para evitar desviaciones costosas.
- Evaluar rutas alternativas en zonas con peajes altos por tonelada transportada.
